<a href="https://colab.research.google.com/github/amitadhainje/CodingAssignment-UBS/blob/master/BERT_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Installing the required modules
!pip install bert-for-tf2
!pip install sentencepiece
print ("The modules like bert-for-tf2 and sentencepiece is installed")

     |████████████████████████████████| 40kB 6.0MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.13.5-cp36-none-any.whl size=29948 sha256=f09310806f83a532b9a7ee36a24c33397148d19e87ef875db12f16b91640c471
  Stored in directory: /root/.cache/pip/wheels/ed/11/48/86aff8a0442aa4728c921027a4100ac28878e9d977e76e2d52
  Created wheel for py-params: filename=py_params-0.8.3-cp36-none-any.whl size=4690 sha256=caf4ace70f4aced87046f4540a0a454bb5dd91fc07da85d22112d4b065363b15
  Stored in directory: /root/.cache/pip/wheels/1a/98/f8/502fc6b5a7c969276f611efeef0d074c871092d38672203b27
  Created wheel for params-flow: filename=params_flow-0.7.4-cp36-none-any.whl size=16195 sha256=4f5d837b67dd382a709640ab03b59b4c480c19af30cc41ed7f34fbc85986bcb5
  Stored in directory: /root/.cache/pip/wheels/86/30/40/507b60d68b67ac87f35e95c98f5b296a32f146d5ae1d1d5aa7
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.0MB 10.8MB/s 
The modules like bert-for-tf2 and 

In [0]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup

In [0]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert


In [6]:
print ("The tensorflow version - ",tf.__version__)
print ("The tensorflow_hub  version - ",hub.__version__)
print ("The pandas version - ",pd.__version__)
print ("The numpy version - ",np.__version__)
#print ("The sentencepiece version - ",pd.__version__)

The tensorflow version -  2.1.0
The tensorflow_hub  version -  0.7.0
The pandas version -  0.25.3
The numpy version -  1.18.1


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [16]:
!ls -ltr /content/drive/My Drive/Colab Notebooks/train.csv

ls: cannot access '/content/drive/My': No such file or directory
ls: cannot access 'Drive/Colab': No such file or directory
ls: cannot access 'Notebooks/train.csv': No such file or directory


In [139]:
columns = ['sentiment', 'id', 'date', 'query','user', 'text']
trainData = pd.read_csv("/content/drive/My Drive/Colab Notebooks/train.csv", header=None, names=columns,
                   engine="python", encoding='latin1')

#positiveData = dataset[dataset['sentiment'] == 4][0:50000]
#negativeData = dataset[dataset['sentiment'] == 0][0:50000]
#frames = [positiveData, negativeData]
#trainData = pd.concat(frames)
trainData.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [140]:
trainData.shape

(1600000, 6)

In [141]:
trainData.drop(['id','date','query','user'], axis=1, inplace=True)
trainData.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [142]:
trainData['sentiment'].value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [0]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, 'lxml').get_text()
    tweet = re.sub(r"@[A-Z-a-z-0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Z-a-z-0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z0-9.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

data_clean = [clean_tweet(tweet) for tweet in trainData.text]

In [144]:
data_clean[0]

" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D"

In [0]:
data_labels = trainData.sentiment.values
data_labels[data_labels == 4] = 1

In [0]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1', trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [147]:
# create tokens from a sentence and convert it to ids.
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("my dog loves strawberries."))

[1139, 3676, 7871, 15235, 22630, 119]

In [0]:
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [0]:
# The training data is sorted according to the target variable, hence we are going to shuffle the data
import random
data_with_len = [[sent,data_labels[i], len(sent)]
                 for i,sent in enumerate(data_inputs)]

random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [(sent_lab[0], sent_lab[1])
                for sent_lab in data_with_len if sent_lab[2] > 7 ]

In [0]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [151]:
next(iter(all_dataset))

(<tf.Tensor: shape=(8,), dtype=int32, numpy=
 array([  178,   112,   182,  1136,   170,   175, 23140,  1825],
       dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)

In [0]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [153]:
next(iter(all_batched))

(<tf.Tensor: shape=(32, 8), dtype=int32, numpy=
 array([[  178,   112,   182,  1136,   170,   175, 23140,  1825],
        [ 2750, 16004, 12204,  1105,  1390, 10151,  4164,  2106],
        [  178,  1567,  1139,  1207,   188,  2522,  5674,  1186],
        [ 9691,  1906,  1111,  1240,  2298,  1120, 13752,   193],
        [  146,  1274,   112,   189, 14425,  2750,   106,   106],
        [ 1486,   181,  9291,  1567, 23684,  1113,  1103,  7159],
        [ 4428, 18080,   119,   119,   119,  1133,  2041,   106],
        [ 1178,   125,  2277,   136,   106,   152, 11215,   106],
        [  146,  1567,  1128,  1177,  1277,  3177, 11006,  3464],
        [ 9326,  4359,   178,  1444,  1234,  1106,  3087,   106],
        [17869,  2490, 22797,   119,  1185,  1141,  1110,  3294],
        [  178,  1138,  1139, 19062, 21907,  2969,  1113,   106],
        [14820,  8685,  1632,  1329,  1104,  3987,  4351,   138],
        [  146,   112,   182,  1136,  6100,  1294,  1122,   119],
        [ 2119,  4477,   106

In [0]:
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE )
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)


In [0]:
class DCNN(tf.keras.Model):

    def __init__(self,
                 vocab_size,
                emb_dim=128,
                 nb_filters=50,# 50 feature dectectors
                 FFN_units=512, # number of hidden units in last 2 layers
                 nb_classes=2,
                 dropout_rate = 0.2,
                 name = "dcnn"):
        super(DCNN, self).__init__(name=name)

        self.embedding = layers.Embedding(vocab_size,
                                            emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters, 
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        
        self.trigram = layers.Conv1D(filters=nb_filters, 
                                    kernel_size=3,
                                    padding="valid",
                                    activation="relu")
        
        self.fourgram = layers.Conv1D(filters=nb_filters, 
                                    kernel_size=4,
                                    padding="valid",
                                    activation="relu")
        
        self.pool = layers.GlobalMaxPooling1D()

        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")

        self.dropout = layers.Dropout(rate=dropout_rate)

        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1, activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes, activation="softmax")

    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)

        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)

        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)

        merged = tf.concat([x_1, x_2, x_3], axis=-1) # batch_size = 3*nb_filters
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output


In [0]:
VOCAB_SIZE =  len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [0]:
dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            dropout_rate=DROPOUT_RATE)

In [0]:
if NB_CLASSES == 2:
    dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=['accuracy'])
else:
    dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=['sparse_categorical_accuracy'])

In [159]:
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/ckpt_bert"

ckpt = tf.train.Checkpoint(Dcnn=dcnn)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ("Latest checkpoint is restored")

Latest checkpoint is restored


In [0]:
class MyCustomCallBack(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print ("Checkpoint is saved at {} ". format(checkpoint_path))


In [0]:
dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallBack()])

Epoch 1/5
  38137/Unknown - 2343s 61ms/step - loss: 0.4246 - accuracy: 0.8056Checkpoint is saved at /content/drive/My Drive/Colab Notebooks/ckpt_bert 
38137/38137 [==============================] - 2343s 61ms/step - loss: 0.4246 - accuracy: 0.8056
Epoch 2/5
22274/38137 [================>.............] - ETA: 16:17 - loss: 0.3565 - accuracy: 0.8444Buffered data was truncated after reaching the output size limit.

In [0]:
results = dcnn.evaluate(test_dataset)
print (results)

In [0]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)
    inputs = tf.expand_dims(tokens, 0)

    output = dcnn(inputs, training=False)
    
    sentiment = math.floor(output*2)

    if(sentiment == 0):
        print ("Output from the model {}.\n Predicted sentiment = negative". format(output))
    elsif(sentiment == 1):
        print ("Output from the model {}.\n Predicted sentiment = postive". format(output))

In [0]:
get_prediction("the movie was very horrible..")